In [1]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression

##### Пусть есть 1000 городов с бредовыми данными по населению, коэффициенту рождаемости и др.

In [101]:
# количество населенных пунктов
n_towns = 1000
# коэффициент рождаемости и население
fertility = np.random.choice(20, n_towns) + 8 + np.random.uniform(low=0, high=1, size=n_towns)
population = np.random.choice(1000000, n_towns) + 1000
# количество понаехавших
plus = np.random.choice(10000, n_towns) + 0
# количество убывших (умерло или уехало)
minus = np.random.choice(10000, n_towns) + 0
# мусорные признаки
number_of_kindergartens = np.random.choice(1000, n_towns) + 1
number_of_schools = np.random.choice(1000, n_towns) + 1
# пусть нужно определить население через год
future = np.round((fertility / 1000) * population + plus - minus + population)

#
data = pd.DataFrame({'fertility': fertility,
                     'population': population,
                     'plus': plus,
                     'minus': minus,
                     'number_of_kindergartens': number_of_kindergartens,
                     'number_of_schools': number_of_schools,
                     'future': future
                    })
data[:5]

,fertility,population,plus,minus,number_of_kindergartens,number_of_schools,future
0,23.881881,32371,1333,1862,65,768,32615.0
1,18.274839,416997,8693,5263,629,669,428048.0
2,22.795141,133259,3037,8478,346,446,130856.0
3,20.712184,945185,7988,9726,99,881,963024.0
4,19.715605,560822,4962,3305,455,111,573536.0


##### Если обучаем модель по всему df

In [102]:
from sklearn.metrics import mean_absolute_error

X = data[['fertility','population','plus','minus','number_of_kindergartens','number_of_schools']]
y = data['future']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['fertility','population','plus','minus','number_of_kindergartens','number_of_schools']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [ 5.09373055e+02  1.01787160e+00  1.00803445e+00 -9.95942580e-01
  1.80179858e-01 -1.57345803e-02]
Bias: -9267.414662253577
Error: 1222.2408956182305


##### Самый большой вклад в предсказание вносит коэффициент рождаемости, что логично.

##### Попробуем убрать последние два признака (с наименьшим весом, заведомо мусорные)

In [105]:
from sklearn.metrics import mean_absolute_error

X = data[['fertility','population','plus','minus']]
y = data['future']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['fertility','population','plus','minus']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [508.73698845   1.01787856   1.00784446  -0.99578619]
Bias: -9177.81153607869
Error: 1220.7225063620583


##### Ещё попытка, без прибыли-убыли

In [106]:
X = data[['population', 'fertility']]
y = data['future']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['population', 'fertility']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [  1.01825763 512.99746928]
Bias: -9311.058742593857
Error: 3570.4781452929956


##### Стало только хуже. Получается, прибыль и убыль убирать нельзя из обучения, хотя они и используются только для расчёта.